## Deliverable 2. Create a Customer Travel Destinations Map.

In [11]:
# Dependencies and Setup
import pandas as pd
import gmaps
import requests
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [12]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Thompson,CA,55.7435,-97.8558,57.36,54,75,11.50,broken clouds
1,1,Abu Dhabi,AE,24.4667,54.3667,100.51,63,19,8.05,few clouds
2,2,Puerto Ayora,EC,-0.7393,-90.3518,78.76,85,38,8.99,scattered clouds
3,3,Albany,US,42.6001,-73.9662,78.73,69,96,5.77,overcast clouds
4,4,Sept-Iles,CA,50.2001,-66.3821,67.24,81,75,10.00,broken clouds


In [13]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your desired minimum temperature for your trip? "))
max_temp = float(input("What is your desired maximum temperature for your trip? "))

In [14]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) &
                                       (city_data_df["Max Temp"] <= max_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Sept-Iles,CA,50.2001,-66.3821,67.24,81,75,10.00,broken clouds
10,10,Tocopilla,CL,-22.0920,-70.1979,66.25,64,98,10.83,overcast clouds
15,15,Valparaiso,CL,-33.0393,-71.6273,60.44,69,75,11.50,broken clouds
38,38,Cape Town,ZA,-33.9258,18.4232,62.28,48,0,2.30,clear sky
40,40,Vostok,RU,46.4856,135.8833,65.86,90,86,4.74,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
649,649,Antofagasta,CL,-23.6500,-70.4000,66.22,60,72,9.62,broken clouds
652,652,Baruun-Urt,MN,46.6806,113.2792,64.51,59,100,13.44,overcast clouds
674,674,Stromness,GB,58.9650,-3.2960,62.56,88,16,12.48,few clouds
675,675,Naushki,RU,50.3828,106.1056,60.89,70,0,5.68,clear sky


In [15]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [16]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Sept-Iles,CA,50.2001,-66.3821,67.24,81,75,10.00,broken clouds
10,10,Tocopilla,CL,-22.0920,-70.1979,66.25,64,98,10.83,overcast clouds
15,15,Valparaiso,CL,-33.0393,-71.6273,60.44,69,75,11.50,broken clouds
38,38,Cape Town,ZA,-33.9258,18.4232,62.28,48,0,2.30,clear sky
40,40,Vostok,RU,46.4856,135.8833,65.86,90,86,4.74,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
649,649,Antofagasta,CL,-23.6500,-70.4000,66.22,60,72,9.62,broken clouds
652,652,Baruun-Urt,MN,46.6806,113.2792,64.51,59,100,13.44,overcast clouds
674,674,Stromness,GB,58.9650,-3.2960,62.56,88,16,12.48,few clouds
675,675,Naushki,RU,50.3828,106.1056,60.89,70,0,5.68,clear sky


In [17]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Sept-Iles,CA,67.24,broken clouds,50.2001,-66.3821,
10,Tocopilla,CL,66.25,overcast clouds,-22.0920,-70.1979,
15,Valparaiso,CL,60.44,broken clouds,-33.0393,-71.6273,
38,Cape Town,ZA,62.28,clear sky,-33.9258,18.4232,
40,Vostok,RU,65.86,overcast clouds,46.4856,135.8833,
44,Vaini,TO,68.16,fog,-21.2000,-175.2000,
47,Faya,SA,66.31,overcast clouds,18.3851,42.4509,
48,Kaa-Khem,RU,65.52,overcast clouds,51.6833,94.7333,
65,Lulea,SE,68.07,clear sky,65.5842,22.1547,
75,San Quintin,MX,68.47,overcast clouds,30.4833,-115.9500,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Cannot find hotel...skipping.")
        

Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.


In [19]:
# 7. Drop the rows where there is no Hotel Name.
c = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = c.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Sept-Iles,CA,67.24,broken clouds,50.2001,-66.3821,Comfort Inn
10,Tocopilla,CL,66.25,overcast clouds,-22.0920,-70.1979,Hotel Galvarino
15,Valparaiso,CL,60.44,broken clouds,-33.0393,-71.6273,Hotel Ultramar
38,Cape Town,ZA,62.28,clear sky,-33.9258,18.4232,Southern Sun Waterfront Cape Town
44,Vaini,TO,68.16,fog,-21.2000,-175.2000,Keleti Beach Resort
...,...,...,...,...,...,...,...
648,Lircay,PE,61.34,clear sky,-12.9896,-74.7205,Hostal Escobar
649,Antofagasta,CL,66.22,broken clouds,-23.6500,-70.4000,"Holiday Inn Express Antofagasta, an IHG Hotel"
652,Baruun-Urt,MN,64.51,overcast clouds,46.6806,113.2792,Ахмадын амралт сувилал
674,Stromness,GB,62.56,few clouds,58.9650,-3.2960,The Old Smokery


In [20]:
# 8a. Create the output File (CSV)
output_data_file = "weatherpy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [21]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [22]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig


Figure(layout=FigureLayout(height='420px'))